In [48]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
from rich.console import Console

console = Console()
print = console.print

In [99]:
data_all = '/Users/codyreece/Desktop/Repos/airbnb/air-scrape/Data/Bentonville.json'
data1 = '/Users/codyreece/Desktop/Repos/airbnb/air-scrape/Data/data_0.json'

with open(data_all) as f_all:
    json_data_all = json.load(f_all)

with open(data1) as f:
    json_data = json.load(f)

print(f'Data type for one listing: {type(json_data)}')
print(f'Data type for all listings: {type(json_data_all)} of {type(json_data)}')
pprint(json_data, sort_dicts=False)
json_data_keys = json_data.keys()
print(json_data_keys)

Data type for one listing: <class 'dict'>

Data type for all listings: <class 'list'> of <class 'dict'>

{'url': 'https://www.airbnb.com/rooms/32721035',
 'name': 'Cozy & Relaxing Suite: Pool/Movie Screen/Trails',
 'stars': 5,
 'numberOfGuests': 4,
 'address': 'Bentonville, Arkansas, United States',
 'roomType': 'Entire guest suite',
 'location': {'lat': 36.40187, 'lng': -94.22518},
 'reviews': [{'author': {'firstName': 'Zach',
                         'hasProfilePic': True,
                         'id': 190646734,
                         'pictureUrl': 'https://a0.muscache.com/im/pictures/user/85dcc9c8-59d6-4f8c-b81b-db3042afc9f0.jpg?aki_policy=profile_x_medium',
                         'smartName': 'Zach',
                         'thumbnailUrl': 'https://a0.muscache.com/im/pictures/user/85dcc9c8-59d6-4f8c-b81b-db3042afc9f0.jpg?aki_policy=profile_small'},
              'authorId': 190646734,
              'canBeEdited': False,
              'comments': 'Enjoyed our stay here! It was in a perfect location '
                          'near downtown and all the biking trails. Clean '
   

dict_keys(['url', 'name', 'stars', 'numberOfGuests', 'address', 'roomType', 'location', 
'reviews', 'pricing', 'calendar', 'listingAmenities', 'rootAmenitySections', 
'seeAllAmenitySections', 'bathroomLabel', 'bedLabel', 'bedroomLabel', 'hasHouseRules', 'id', 
'isNewListing', 'listingRooms', 'p3SummaryAddress', 'photos', 'primaryHost', 
'roomTypeCategory', 'sectionedDescription', 'tierId', 'guestControls', 'hasSpecialOffer', 
'maxNights', 'minNights', 'priceDetails', 'hasWeWorkLocation', 'isBusinessTravelReady', 
'additionalHosts', 'isHostedBySuperhost', 'hometourRooms', 'hometourSections', 
'descriptionLocale', 'initialDescriptionAuthorType', 'localizedCheckInTimeWindow', 
'localizedCheckOutTime', 'city', 'country', 'countryCode', 'hasHostGuidebook', 
'hasLocalAttractions', 'neighborhoodCommunityTags', 'state', 
'paidGrowthRemarketingListingIds', 'hasCommercialHostInfo', 'reviewDetailsInterface', 
'reviewsOrder', 'hostQuote', 'hostSignatureFont', 'nearbyAirportDistanceDescriptions', 
'propertyTypeInCity', 'renderTierId', 'isHotel', 'showReviewTag', 
'isRepresentativeInventory', 'localizedCity', 'highlights', 'highlightsImpressionId', 
'categorizedPreviewAmenities', 'pointOfInterests', 'chinaPointsOfInterestMatcha', 
'hostGuidebook', 'pageViewType', 'previewTags', 'seeAllHometourSections', 'educationModules',
'enableHighlightsVoting', 'listingExpectations', 'localizedListingExpectations', 
'heroModule', 'reviewsModule', 'availabilityModule', 'additionalHouseRules', 'sortedReviews',
'documentDisplayPictures', 'sections', 'metadata', 'p3ImpressionId', 'idStr', 
'paidGrowthRemarketingListingIdsStr'])

In [93]:
columns=['idStr', 'stars', 'numberOfGuests', 'bathroomLabel', 'bedLabel', 'bedroomLabel', 'city', 'roomTypeCategory']

json_data_to_df = pd.DataFrame(json_data, columns=columns, index=[0])

In [94]:
json_data_to_df

,idStr,stars,numberOfGuests,bathroomLabel,bedLabel,bedroomLabel,city,roomTypeCategory
0,32721035,5,4,1 bath,2 beds,1 bedroom,Bentonville,entire_home
